## 1. A Simple Workload consisting of a Bag-of-Tasks Remote Execution

In [1]:
import os,sys
import radical.pilot as rp

os.environ["RADICAL_PILOT_DBURL"]="mongodb://ec2-54-221-194-147.compute-1.amazonaws.com:24242/sc15tut"

#### Create a Radical Pilot Session

In [2]:
session = rp.Session()
print "Session id %s"%session.uid

Session id rp.session.radical-1.iparask.016739.0006


#### Create a Radical Pilot context to include your credentials for the Remote Target Machine

In [3]:
c = rp.Context('ssh')
c.user_id = ""
session.add_context(c)

#### Submit the Session to a Pilot Manager

In [4]:
print "Initializing Pilot Manager ..."
pmgr = rp.PilotManager(session=session)

Initializing Pilot Manager ...


#### Create a Pilot description and submit the pilot to the pilot manager

In [5]:
pdesc = rp.ComputePilotDescription ()
pdesc.resource = "xsede.stampede"  # NOTE: This is a "label", not a hostname
pdesc.runtime  = 10 # minutes
pdesc.cores    = 16
pdesc.cleanup  = False
pdesc.project  = ''
pdesc.queue    = 'normal'
# submit the pilot.
print "Submitting Compute Pilot to Pilot Manager ..."
pilot = pmgr.submit_pilots(pdesc)

Submitting Compute Pilot to Pilot Manager ...


#### Combine the ComputePilot, the ComputeUnits and a scheduler via a UnitManager object.

In [6]:
print "Initializing Unit Manager ..."
umgr = rp.UnitManager (session=session,
                           scheduler=rp.SCHED_DIRECT_SUBMISSION)

Initializing Unit Manager ...


#### Add the created ComputePilot to the UnitManager.

In [7]:
print "Registering Compute Pilot with Unit Manager ..."
umgr.add_pilots(pilot)

Registering Compute Pilot with Unit Manager ...


#### Create the description of the tasks to be run

In [8]:
NUMBER_JOBS  = 64 # the total number of cus to run
# submit CUs to pilot job
cudesc_list = []
for i in range(NUMBER_JOBS):
    # -------- BEGIN USER DEFINED CU DESCRIPTION --------- #
    cudesc = rp.ComputeUnitDescription()
    cudesc.environment = {'CU_NO': i}
    cudesc.executable  = "/bin/echo"
    cudesc.arguments   = ['I am CU number $CU_NO']
    cudesc.cores       = 1
    # -------- END USER DEFINED CU DESCRIPTION --------- #
    cudesc_list.append(cudesc)

#### Submit the previously created ComputeUnit descriptions to the
#### PilotManager. This will trigger the selected scheduler to start
#### assigning ComputeUnits to the ComputePilots.
 
#### Wait until all the tasks are complete

In [9]:

print "Submit Compute Units to Unit Manager ..."
cu_set = umgr.submit_units (cudesc_list)
print "Waiting for CUs to complete ..."
umgr.wait_units()

Submit Compute Units to Unit Manager ...
Waiting for CUs to complete ...


[u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done',
 u'Done']

#### Printing tasks final state

In [10]:
print "All CUs completed successfully!"
for unit in cu_set:
    print "* CU %s, state %s, exit code: %s, stdout: %s" \
     % (unit.uid, unit.state, unit.exit_code, unit.stdout)

All CUs completed successfully!
* CU unit.000000, state Done, exit code: 0, stdout: I am CU number 0

* CU unit.000001, state Done, exit code: 0, stdout: I am CU number 1

* CU unit.000002, state Done, exit code: 0, stdout: I am CU number 2

* CU unit.000003, state Done, exit code: 0, stdout: I am CU number 3

* CU unit.000004, state Done, exit code: 0, stdout: I am CU number 4

* CU unit.000005, state Done, exit code: 0, stdout: I am CU number 5

* CU unit.000006, state Done, exit code: 0, stdout: I am CU number 6

* CU unit.000007, state Done, exit code: 0, stdout: I am CU number 7

* CU unit.000008, state Done, exit code: 0, stdout: I am CU number 8

* CU unit.000009, state Done, exit code: 0, stdout: I am CU number 9

* CU unit.000010, state Done, exit code: 0, stdout: I am CU number 10

* CU unit.000011, state Done, exit code: 0, stdout: I am CU number 11

* CU unit.000012, state Done, exit code: 0, stdout: I am CU number 12

* CU unit.000013, state Done, exit code: 0, stdout: I a

#### Always clean up the session

In [11]:
print "closing session"
session.close ()

closing session
